# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,11.39,89,100,1.92,NZ,1699815329
1,1,grytviken,-54.2811,-36.5092,5.46,81,100,2.51,GS,1699815329
2,2,albany,42.6001,-73.9662,6.00,55,10,1.34,US,1699815207
3,3,invercargill,-46.4000,168.3500,11.68,95,99,2.68,NZ,1699815329
4,4,port alfred,-33.5906,26.8910,13.99,85,6,0.89,ZA,1699815329


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_points_simple = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles='ESRI' #ESRI is used because it looks neat.
)

# Display the map

map_points_simple

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
filtered_df = city_data_df[
        (city_data_df["Max Temp"] >= 20) & (city_data_df["Max Temp"] <= 30) &
        (city_data_df["Humidity"] <= 100) &
        (city_data_df["Cloudiness"] <=50) &
        (city_data_df["Wind Speed"] <= 15) &
        (city_data_df["Country"].isin(['NZ', 'US', 'AU', 'MX','CA','EC','RU']))
        
]
# Drop any rows with null values
filtered_df=filtered_df.dropna()

# Display sample data
filtered_df.head(40)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,carnarvon,-24.8667,113.6333,20.65,83,23,3.37,AU,1699815332
36,36,cabo san lucas,22.8909,-109.9124,29.78,59,12,2.24,MX,1699815336
40,40,hawaiian paradise park,19.5933,-154.9731,26.55,77,47,0.45,US,1699815337
82,82,puerto baquerizo moreno,-0.9000,-89.6000,28.55,61,40,8.23,EC,1699815346
105,105,karratha,-20.7377,116.8463,22.38,64,6,6.53,AU,1699815350
169,169,puerto ayora,-0.7393,-90.3518,28.98,88,34,3.58,EC,1699815359
172,172,lompoc,34.6391,-120.4579,26.99,38,12,5.81,US,1699815199
178,178,hilo,19.7297,-155.0900,27.01,70,21,1.79,US,1699815360
236,236,port douglas,-16.4833,145.4667,22.80,78,43,3.58,AU,1699815375
251,251,artesia,33.8658,-118.0831,29.85,17,0,3.09,US,1699815221


In [19]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17 entries, 16 to 538
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     17 non-null     int64  
 1   City        17 non-null     object 
 2   Lat         17 non-null     float64
 3   Lng         17 non-null     float64
 4   Max Temp    17 non-null     float64
 5   Humidity    17 non-null     int64  
 6   Cloudiness  17 non-null     int64  
 7   Wind Speed  17 non-null     float64
 8   Country     17 non-null     object 
 9   Date        17 non-null     int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 1.5+ KB


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df.copy()
hotel_df = hotel_df.reindex(columns=["City","Country","Lat","Lng","Humidity"])
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"]=None
# Display sample data
# YOUR CODE HERE
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
16,carnarvon,AU,-24.8667,113.6333,83,None
36,cabo san lucas,MX,22.8909,-109.9124,59,None
40,hawaiian paradise park,US,19.5933,-154.9731,77,None
82,puerto baquerizo moreno,EC,-0.9000,-89.6000,61,None
105,karratha,AU,-20.7377,116.8463,64,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
latitude=22.0752
longitude=-159.3190
radius=10000
limit=1
###Note:Search places inside of the circle. Format (circle:lon,lat,radiusMeters)
filters = f"circle:{longitude},{latitude},{radius}" 
###Note:Prioritize results by farness from the location. Format bias=proximity:lon,lat
bias = f"proximity:{longitude},{latitude}" 
params = {
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude=hotel_df.loc[index, "Lat"]
    longitude=hotel_df.loc[index, "Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}" 
    params["categories"] = f"accommodation.hotel"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    #print(name_address)
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
carnarvon - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
hawaiian paradise park - nearest hotel: No hotel found
puerto baquerizo moreno - nearest hotel: Dory's House
karratha - nearest hotel: Karratha International Hotel
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
hilo - nearest hotel: Dolphin Bay Hotel
port douglas - nearest hotel: Mantra Aqueous Port Douglas
artesia - nearest hotel: Days Inn & Suites by Wyndham Artesia
holualoa - nearest hotel: Kona Hotel
kailua-kona - nearest hotel: Kona Seaside Hotel
heroica guaymas - nearest hotel: Hotel Playa de Cortéz
guerrero negro - nearest hotel: Plaza sal paraiso
mount juliet - nearest hotel: Residence Inn
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
pacific grove - nearest hotel: Pacific Grove Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
16,carnarvon,AU,-24.8667,113.6333,83,No hotel found
36,cabo san lucas,MX,22.8909,-109.9124,59,Comfort Rooms
40,hawaiian paradise park,US,19.5933,-154.9731,77,No hotel found
82,puerto baquerizo moreno,EC,-0.9000,-89.6000,61,Dory's House
105,karratha,AU,-20.7377,116.8463,64,Karratha International Hotel
169,puerto ayora,EC,-0.7393,-90.3518,88,Hostal La Mirada De Solitario George
172,lompoc,US,34.6391,-120.4579,38,Embassy Suites by Hilton Lompoc Central Coast
178,hilo,US,19.7297,-155.0900,70,Dolphin Bay Hotel
236,port douglas,AU,-16.4833,145.4667,78,Mantra Aqueous Port Douglas
251,artesia,US,33.8658,-118.0831,17,Days Inn & Suites by Wyndham Artesia


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Create a dictionary containing coordinates

map_points_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles='OSM', #"OSM", was used over, "ESRI", from the previous map, because the points blended in too much
    frame_width = 700,
    frame_height = 500,
    size='Humidity',
    color="Country",
    hover_cols=["City", "Hotel Name", "Address"]
)
# Display the map
map_points_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,City,Hotel Name)